In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import arviz as az
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pystan
%load_ext jupyterstan

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
STAN_MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "stan_models")
STAN_DATA_PATH = os.path.join(PROJECT_ROOT_DIR,'data')

from stan_utils import *

## 准备数据

In [14]:
parameter = az.from_netcdf('experement_3.nc')

In [39]:
parameter.posterior.gama.values.reshape(-1).tolist()

[3.501249722572016,
 3.4981618509094714,
 3.5029594015825656,
 3.5010102621859724,
 3.5021666917604715,
 3.5008610114924856,
 3.4985622321366425,
 3.4937037506606874,
 3.4962595978318594,
 3.4969901054733703,
 3.4975710983209467,
 3.501636894989507,
 3.5040777997457573,
 3.4950406866068047,
 3.4990744048570326,
 3.507678458541253,
 3.4933915689768935,
 3.4912773647069097,
 3.4961712150117608,
 3.4945273393275915,
 3.497969217849248,
 3.499976548257201,
 3.4963136608115097,
 3.4936570507344378,
 3.505744480817763,
 3.4926794515929673,
 3.4940997276085914,
 3.4982196951389097,
 3.5062837785809218,
 3.503278360887997,
 3.501265490265142,
 3.502516551428872,
 3.498542904239259,
 3.4979081748139795,
 3.498976680123523,
 3.4958123135259447,
 3.504771447078316,
 3.4932116377196767,
 3.5104590239657347,
 3.490570817277357,
 3.5001696075654234,
 3.5050510962561114,
 3.4973266813536417,
 3.4961051950900988,
 3.5069335216891906,
 3.506872718512698,
 3.4958082405568125,
 3.49596657197417,
 3.49987

In [5]:
def gen_stan_data_for_predict(data, pred_group, parameter_samples):
    '''
    20×20 其中10
     Parameters
        ----------
        data:dict
              'trans_data': xarray格式,dim = (group, var:[x,y,power]),每组对应一个发送点的x，y坐标和发送功率值
              ’true_recv_power‘: xarray格式，dim=('group','y', 'x'),表示对应trans_data某组坐标（x，y）处的接收功率，不考虑误差
              ’sensored_power':xarray格式，dim=('group','y', 'x')，表示在（x，y）处设置感知设备的接收功率，考虑误差
              
        pred_group:选取data中的哪一组进行预测
        
        parameter_samples：arvizdata.posterior
     Returns
        ----------
        stan_data:
              int<lower=1> N_pred; //需要预测的点个数
              vector[N_pred] pred_coord_x;// 每组数据有N个感知节点
              vector[N_pred] pred_coord_y;// 
              int<lower=1> N;
              vector[N] coord_x;
              vector[N] coord_y;
              vector[N] Y_observe;
              real trans_power; //信号源发送功率
              real trans_x; //信号源坐标
              real trans_y;

              int <lower=1> N_samples;
              vector[N_samples] gama; // 自由空间传播损耗因子
              vector[N_samples] fai;  // matern 协方差函数的距离 参数
              vector[N_samples] sigma; // 空间方差  
              vector[N_samples] sigma_eta; // 测量误差
    '''
    trans_data = data['trans_data'][pred_group,:]
    sensored_data_of_sensors = data['sensored_power'][pred_group,::2, ::2] # 隔一个取
    trued_data_for_predict = data['true_recv_power'][pred_group, 1::2, 1::2] #隔一个取作为预测数据的参考值
    stan_data={}
    
    stan_data['N_pred'] = trued_data_for_predict.shape[0] * trued_data_for_predict.shape[1]
    stan_data['N'] = sensored_data_of_sensors.shape[0] * sensored_data_of_sensors.shape[1]
    
    x_pred,y_pred = np.meshgrid(trued_data_for_predict.x.values, trued_data_for_predict.y.values)
    stan_data['pred_coord_x'] = list(x_pred.flatten())
    stan_data['pred_coord_y'] = list(y_pred.flatten())
    
    x,y = np.meshgrid(sensored_data_of_sensors.x.values, sensored_data_of_sensors.y.values)
    stan_data['coord_x'] = list(x.flatten())
    stan_data['coord_y'] = list(y.flatten())
    
    stan_data['Y_observe'] = list(sensored_data_of_sensors.values.reshape(-1))
    
    stan_data['trans_power'] = trans_data.sel(var='power').values.tolist()
    stan_data['trans_x'] = trans_data.sel(var='x').values.tolist()
    stan_data['trans_y'] = trans_data.sel(var='y').values.tolist()
    
    stan_data['N_samples'] = parameter_samples.sizes['chain'] * parameter_samples.sizes['draw']
    stan_data['gama'] = list(parameter_samples.gama.values.reshape(-1))
    stan_data['fai'] = list(parameter_samples.fai.values.reshape(-1))
    stan_data['sigma'] = list(parameter_samples.sigma.values.reshape(-1))
    stan_data['sigma_eta'] = list(parameter_samples.sigma_eta.values.reshape(-1))
    return stan_data

## 20组数据的结果

In [32]:
data_20groups = StanData_load('data_20groups')
experiment2 = az.from_netcdf('experement_2.nc')
parameter_samples_20groups = experiment2.posterior

Using cached StanDATA: data_20groups


In [43]:
stan_data_pred_20groups = gen_stan_data_for_predict(data_20groups, 1 , parameter_samples_20groups)

In [45]:
StanData_cache(stan_data_pred_20groups,'stan_data_pred_20groups')

DATA cached as:stan_data_pred_20groups.pkl


## 1000组数据获取的参数的结果

In [3]:
data_20groups = StanData_load('data_20groups')
experiment3 = az.from_netcdf('experement_3.nc')
parameter_samples_1000groups = experiment3.posterior

Using cached StanDATA: data_20groups


In [6]:
stan_data_pred_1000para = gen_stan_data_for_predict(data_20groups, 1 , parameter_samples_1000groups)

In [7]:
StanData_cache(stan_data_pred_1000para,'stan_data_pred_1000para')

DATA cached as:stan_data_pred_1000para.pkl
